In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
import tpot
import timeit
import pywt 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.pipeline import make_pipeline
from tpot.builtins import StackingEstimator
import sklearn.metrics

plt.rcParams.update({'figure.max_open_warning': 0})

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [3]:
df_train = pd.read_csv(r'C:\Users\Firdaus Ridzuan\Downloads\Data3_1_2020\train.csv', na_values = -999)
df_train.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
0,NaN,NaN,41.4699,NaN,NaN,NaN,NaN,128.0737,319.0654
1,NaN,NaN,42.5053,NaN,NaN,NaN,NaN,127.8347,318.7825
2,NaN,NaN,43.1548,NaN,NaN,NaN,NaN,127.2307,317.3323
3,NaN,NaN,43.2410,NaN,NaN,NaN,NaN,126.2917,313.6486
4,NaN,NaN,40.3218,NaN,NaN,NaN,NaN,125.3985,307.8903


In [4]:
df_test = pd.read_csv(r'C:\Users\Firdaus Ridzuan\Downloads\Data3_1_2020\test.csv', na_values = -999)
df_test.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN
0,8.5781,0.3521,55.1824,0.8121,0.7810,6.8291,2.3256
1,8.5781,0.3639,57.0114,0.8038,0.7723,6.8100,2.3255
2,8.5781,0.3703,58.9263,0.7444,0.7048,6.7766,2.3212
3,8.5625,0.3667,57.3308,0.7169,0.6542,6.7219,2.3119
4,8.5781,0.3500,53.0624,0.6845,0.6109,6.6384,2.2982


In [5]:
# Separate train data based on target
df_tr_dtc = df_train[df_train.DTC.notna()].copy().drop(columns=["DTS"])
df_tr_dts = df_train[df_train.DTS.notna()].copy().drop(columns=["DTC"])

# df with complete log for each DTC and DTS
df_trc_dtc = df_tr_dtc.dropna()
df_trc_dts = df_tr_dts.dropna()

In [6]:
df_tr_dtc.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC
0,NaN,NaN,41.4699,NaN,NaN,NaN,NaN,128.0737
1,NaN,NaN,42.5053,NaN,NaN,NaN,NaN,127.8347
2,NaN,NaN,43.1548,NaN,NaN,NaN,NaN,127.2307
3,NaN,NaN,43.2410,NaN,NaN,NaN,NaN,126.2917
4,NaN,NaN,40.3218,NaN,NaN,NaN,NaN,125.3985


# Feature Engineering
We apply feature windows augmentation (after Bestagini et al, 2017) and multi-level wavelet decomposition to emulate different geologic bed thicknesses.

In [7]:
# wavelet decomposition
def lowpassfilter(signal, thresh = 0, wavelet="db9", level=1):
    thresh = thresh*np.nanmax(signal)
    coeff = pywt.wavedec(signal, wavelet, mode="smooth", level=level)
    coeff[1:] = (pywt.threshold(i, value=thresh, mode="soft" ) for i in coeff[1:])
    reconstructed_signal = pywt.waverec(coeff, wavelet, mode="smooth")
    
    if len(signal) == len(reconstructed_signal):
        return reconstructed_signal
    elif len(signal) == len(reconstructed_signal)-1:
        return reconstructed_signal[:-1]
    else:
        return print('shape mismatch')

In [8]:
# Feature windows augmentation (after Bestagini et al, 2017: https://doi.org/10.1190/segam2017-17729805.1)
def augment_features(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug

# # Feature gradient computation function
# def augment_features_gradient(X, depth):
    
#     # Compute features gradient
#     d_diff = np.diff(depth).reshape((-1, 1))
#     d_diff[d_diff==0] = 0.001
#     X_diff = np.diff(X, axis=0)
#     X_grad = X_diff / d_diff
        
#     # Compensate for last missing value
#     X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
#     return X_grad

# # Feature augmentation function
# def augment_features(X, well, depth, N_neig=1):
    
#     # Augment features
#     X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
#     for w in np.unique(well):
#         w_idx = np.where(well == w)[0]
#         X_aug_win = augment_features_window(X[w_idx, :], N_neig)
#         X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
#         X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
#     # Find padded rows
#     padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
#     return X_aug, padded_rows

# Feature augmentation function
# def augment_features(X, depth, N_neig=1):
    
#     # Augment features
#     X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
#     X_aug_win = augment_features_window(X, N_neig)
#     X_aug_grad = augment_features_gradient(X, depth)
#     X_aug = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
#     # Find padded rows
#     padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
#     return X_aug, padded_rows

In [9]:
# multi-level wavelet decomposition
def apply_fe_simple(df: pd.DataFrame) -> pd.DataFrame:
    cols_in = df.columns.to_list()
    cols_target = ['DTC', 'DTS']
    cols_in = [i for i in cols_in if i not in cols_target]
    
    df_fe = df[cols_in]

    df_fe["HRD_log10"] = np.log10(df_fe.HRD)
    df_fe["HRM_log10"] = np.log10(df_fe.HRM)

    df_fe['GR_db1'] = lowpassfilter(df_fe.GR, thresh = 0.5, wavelet = 'db9', level = 1)
    df_fe['GR_db2'] = lowpassfilter(df_fe.GR, thresh = 0.5, wavelet = 'db9', level = 2)
    df_fe['GR_db3'] = lowpassfilter(df_fe.GR, thresh = 0.5, wavelet = 'db9', level = 3)
    df_fe['GR_db4'] = lowpassfilter(df_fe.GR, thresh = 0.5, wavelet = 'db9', level = 4)
    df_fe['GR_db5'] = lowpassfilter(df_fe.GR, thresh = 0.5, wavelet = 'db9', level = 5)
    df_fe['GR_db6'] = lowpassfilter(df_fe.GR, thresh = 0.7, wavelet = 'db9', level = 6)

    df_fe['HRD_db1'] = lowpassfilter(df_fe.HRD, thresh = 0.5, wavelet = 'db9', level = 1)
    df_fe['HRD_db2'] = lowpassfilter(df_fe.HRD, thresh = 0.5, wavelet = 'db9', level = 2)
    df_fe['HRD_db3'] = lowpassfilter(df_fe.HRD, thresh = 0.5, wavelet = 'db9', level = 3)
    df_fe['HRD_db4'] = lowpassfilter(df_fe.HRD, thresh = 0.5, wavelet = 'db9', level = 4)
    df_fe['HRD_db5'] = lowpassfilter(df_fe.HRD, thresh = 0.5, wavelet = 'db9', level = 5)
    df_fe['HRD_db6'] = lowpassfilter(df_fe.HRD, thresh = 0.7, wavelet = 'db9', level = 6)

    df_fe['HRM_db1'] = lowpassfilter(df_fe.HRM, thresh = 0.5, wavelet = 'db9', level = 1)
    df_fe['HRM_db2'] = lowpassfilter(df_fe.HRM, thresh = 0.5, wavelet = 'db9', level = 2)
    df_fe['HRM_db3'] = lowpassfilter(df_fe.HRM, thresh = 0.5, wavelet = 'db9', level = 3)
    df_fe['HRM_db4'] = lowpassfilter(df_fe.HRM, thresh = 0.5, wavelet = 'db9', level = 4)
    df_fe['HRM_db5'] = lowpassfilter(df_fe.HRM, thresh = 0.5, wavelet = 'db9', level = 5)
    df_fe['HRM_db6'] = lowpassfilter(df_fe.HRM, thresh = 0.7, wavelet = 'db9', level = 6)

    df_fe['ZDEN_db1'] = lowpassfilter(df_fe.ZDEN, thresh = 0.5, wavelet = 'db9', level = 1)
    df_fe['ZDEN_db2'] = lowpassfilter(df_fe.ZDEN, thresh = 0.5, wavelet = 'db9', level = 2)
    df_fe['ZDEN_db3'] = lowpassfilter(df_fe.ZDEN, thresh = 0.5, wavelet = 'db9', level = 3)
    df_fe['ZDEN_db4'] = lowpassfilter(df_fe.ZDEN, thresh = 0.5, wavelet = 'db9', level = 4)
    df_fe['ZDEN_db5'] = lowpassfilter(df_fe.ZDEN, thresh = 0.5, wavelet = 'db9', level = 5)
    df_fe['ZDEN_db6'] = lowpassfilter(df_fe.ZDEN, thresh = 0.7, wavelet = 'db9', level = 6)

    df_fe['CNC_db1'] = lowpassfilter(df_fe.CNC, thresh = 0.5, wavelet = 'db9', level = 1)
    df_fe['CNC_db2'] = lowpassfilter(df_fe.CNC, thresh = 0.5, wavelet = 'db9', level = 2)
    df_fe['CNC_db3'] = lowpassfilter(df_fe.CNC, thresh = 0.5, wavelet = 'db9', level = 3)
    df_fe['CNC_db4'] = lowpassfilter(df_fe.CNC, thresh = 0.5, wavelet = 'db9', level = 4)
    df_fe['CNC_db5'] = lowpassfilter(df_fe.CNC, thresh = 0.5, wavelet = 'db9', level = 5)
    df_fe['CNC_db6'] = lowpassfilter(df_fe.CNC, thresh = 0.7, wavelet = 'db9', level = 6)

    df_fe['CAL_db1'] = lowpassfilter(df_fe.CAL, thresh = 0.5, wavelet = 'db9', level = 1)
    df_fe['CAL_db2'] = lowpassfilter(df_fe.CAL, thresh = 0.5, wavelet = 'db9', level = 2)
    df_fe['CAL_db3'] = lowpassfilter(df_fe.CAL, thresh = 0.5, wavelet = 'db9', level = 3)
    df_fe['CAL_db4'] = lowpassfilter(df_fe.CAL, thresh = 0.5, wavelet = 'db9', level = 4)
    df_fe['CAL_db5'] = lowpassfilter(df_fe.CAL, thresh = 0.5, wavelet = 'db9', level = 5)
    df_fe['CAL_db6'] = lowpassfilter(df_fe.CAL, thresh = 0.7, wavelet = 'db9', level = 6)

    df_best = pd.DataFrame(augment_features(df_fe, N_neig=1))

    df_fe = np.concatenate((df_fe, df_best), axis = 1)

    return pd.DataFrame(df_fe)

In [10]:
X_dtc = apply_fe_simple(df_trc_dtc)
scaler_dtc = RobustScaler().fit(X_dtc)
X_dtc = scaler_dtc.transform(X_dtc)

y_dtc = df_trc_dtc['DTC']

X_dts = apply_fe_simple(df_trc_dts)
scaler_dts = RobustScaler().fit(X_dts)
X_dts = scaler_dts.transform(X_dts)

y_dts = df_trc_dts['DTS']

In [11]:
# Obtained from first TPOT submission using DTC data
rgs_dtc = make_pipeline(
    StackingEstimator(
        estimator=
        ExtraTreesRegressor(bootstrap=False, max_features=0.4,
                            min_samples_leaf=1, min_samples_split=10,
                            n_estimators=100, random_state=24, n_jobs=6)),
    RandomForestRegressor(bootstrap=True, max_features=0.25, min_samples_leaf=2,
                          min_samples_split=20, n_estimators=100, n_jobs=6,
                          random_state=24)
)

# Obtained from first TPOT submission using DTS data
rgs_dts = RandomForestRegressor(bootstrap=True, max_features=0.4,
                                min_samples_leaf=1, min_samples_split=6,
                                n_estimators=100, random_state=24, n_jobs=6)


In [12]:
# Model training
rgs_dtc.fit(X_dtc, y_dtc)
rgs_dts.fit(X_dts, y_dts)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=0.4, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=6, oob_score=False,
                      random_state=24, verbose=0, warm_start=False)

In [13]:
# Model prediction on test data
X_test_dtc = apply_fe_simple(df_test).values
X_test_dtc = scaler_dtc.transform(X_test_dtc)
df_test["DTC"] = rgs_dtc.predict(X_test_dtc)

X_test_dts = apply_fe_simple(df_test).values
X_test_dts = scaler_dts.transform(X_test_dts)
df_test["DTS"] = rgs_dts.predict(X_test_dts)

df_test[["DTC", "DTS"]].to_csv("sample_submission.csv", index=False)